In this notebook I will make 2nd solving algorithm (see readme) - it isn't done yet

In [1]:
import numpy as np
import threading
import sys

In [2]:
#sudoku = np.zeros(81,dtype="int").reshape(9,9)
def clear_sudoku():
    sudoku = [[0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0]]

    sudoku = np.array(sudoku)
    return sudoku

In [3]:
def example_easy_sudoku():
    sudoku = [[0, 7, 0, 4, 0, 5, 6, 0, 0],
              [9, 2, 4, 0, 6, 0, 8, 0, 0],
              [1, 0, 5, 0, 0, 8, 7, 0, 0],
              [0, 3, 8, 0, 5, 7, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 7, 3],
              [0, 4, 7, 0, 9, 0, 1, 2, 0],
              [4, 0, 9, 6, 0, 2, 0, 0, 1],
              [7, 0, 0, 8, 3, 0, 4, 0, 9],
              [0, 1, 0, 5, 4, 9, 0, 0, 0]]

    sudoku = np.array(sudoku)
    return sudoku

In [4]:
def example_hard_sudoku():
    sudoku = [[8, 3, 2, 1, 0, 7, 0, 9, 6],
              [0, 0, 0, 0, 0, 0, 0, 5, 0],
              [7, 4, 5, 0, 9, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 7],
              [0, 0, 0, 8, 0, 1, 0, 0, 0],
              [0, 0, 4, 0, 0, 0, 5, 0, 1],
              [3, 0, 0, 0, 0, 5, 0, 6, 2],
              [0, 6, 0, 0, 0, 8, 0, 0, 4],
              [0, 0, 0, 7, 0, 3, 0, 8, 0]]

    sudoku = np.array(sudoku)
    return sudoku

In [5]:
def example_very_hard_sudoku():
    sudoku = [[3, 7, 0, 0, 0, 0, 0, 9, 0],
              [9, 0, 0, 0, 7, 0, 0, 0, 0],
              [0, 0, 0, 4, 2, 0, 0, 0, 6],
              [0, 0, 1, 0, 8, 4, 2, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0],
              [8, 0, 0, 6, 0, 0, 0, 5, 0],
              [0, 0, 6, 0, 0, 2, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 3, 9],
              [0, 2, 0, 0, 0, 0, 4, 0, 0]]

    sudoku = np.array(sudoku)
    return sudoku

In [6]:
#sudoku[1,3] - value from row 1 and col 3
#sudoku[:,3] - whole column 3
#sudoku[1,:] - whole row 1

In [7]:
def if_possible(sudoku, row, col, number):
    possible = False
    
    ##check if number in row
    if number not in sudoku[row,:]:
        possible = True
    else:
        return False
    
    ##check if number in column
    if number not in sudoku[:,col]:
        possible = True
    else:
        return False
    
    ##check if number in squere 3x3
    start_sq_row = row // 3 #floor qoutient
    # can make it prettier
    if start_sq_row == 1:
        start_sq_row = 3
    elif start_sq_row ==2:
        start_sq_row = 6
    start_sq_col = col // 3
    if start_sq_col == 1:
        start_sq_col = 3
    elif start_sq_col ==2:
        start_sq_col = 6
        
    end_sq_row = start_sq_row + 3
    end_sq_col = start_sq_col + 3
    
#     print("Starting row of squere: {}".format(start_sq_row))
#     print("Starting column of squere: {}".format(start_sq_col))
#     print("Ending row of squere: {}".format(end_sq_row))
#     print("Ending column of squere: {}".format(end_sq_col))
    
    if number not in sudoku[start_sq_row:end_sq_row,start_sq_col:end_sq_col]:
        possible = True
    else:
        return False
    
    return possible

In [8]:
## checking if function works
#if_possible(sudoku, 1, 6, 1)

In [9]:
## just putting in (biggest - for now) possible number in every place

# i for rows
# for i in range(9):
#     # j for cols
#     for j in range(9):
#         # do only if number is 0 
#         if sudoku[i][j] == 0:
#             # check all numbers
#             for n in range(1,10):
#                 # if possible put number there
#                 if if_possible(sudoku, i, j, n) == True:
#                     sudoku[i][j] = n

In [10]:
def restart_next_cells(sudoku, on_start, row, col):
    for i in range(row,9):
        for j in range(9):
            if on_start[i][j] == 0:
                if i == row:
                    if j >= col:
                        sudoku[i][j] = 0
                else:
                    sudoku[i][j] = 0
    return sudoku

In [11]:
## recursive function solving sudoku
# args: sudoku in actual state, row and col that is checked (and filled), number that is checked in this step
def solve(sudoku, on_start, row, col, number, go_back,rec):
    if rec % 1000 == 0:
        print("Recursion: {}".format(rec))
    #if going back and number wasn't filled on start
    if go_back == True and on_start[row][col] == 0:
        #increment number 
        number = sudoku[row][col] + 1
        #if number > 9 error
        if number > 9:
            #INSTEAD OF ERROR NOW I NEED TO PUT HERE MECHANISM THAT WILL GO BACK AND 
            #TRY TO PUT ANOTHER POSSIBLE NUMBER IN PREVIOUS CELL THAT WASN'T FILLED ON START
            #WILL DO IT USING ON_START VARIABLE
            #print("need to go back")
            sudoku = restart_next_cells(sudoku, on_start, row, col)
            if col == 0:
                if row == 0:
                    #print("error - can't go back")
                    #print("return sudoku")
                    print("Recursion: {}".format(rec))
                    return sudoku
                else:
                    col = 8
                    row = row - 1
            else:
                col = col - 1
            
            #print("return solve(sudoku = {}, row = {}, col = {}, number = {}, go_back = {}".format(sudoku,row,col,0,True))
            return solve(sudoku, on_start, row, col, 0, go_back=True,rec = rec + 1)
        #if number <= 9 check if possible and put number in cell 
        else:
            #if possible to put there number incremented 
            if_possib = if_possible(sudoku, row, col, number)
            if if_possib == True:
                #put number in cell
                sudoku[row][col] = number
                #####
                #if last column
                if col == 8:
                    #if last row
                    if row == 8:
                        #we are in last cell and it is not equal to 0 -> return sudoku
                        #print("return sudoku")
                        print("Recursion: {}".format(rec))
                        return sudoku
                    #if not last row
                    else:
                        #go to firs (0) column and next row
                        col = 0
                        row = row + 1
                #if not last column
                else:
                    #go to next column and stay in same row
                    col = col + 1
                #GO TO NEXT CELL #recursive solve with actual state of sudoku, changed row and col and number 1 - because checking values in new cell starts from 1
                #print("return solve(sudoku = {}, row = {}, col = {}, number = {}, go_back = {}".format(sudoku,row,col,1,False))
                return solve(sudoku, on_start, row, col, 1, go_back=False,rec = rec + 1)
                #####
            else:
                #if number is 9 - error
                if number > 9:
                    #INSTEAD OF ERROR NOW I NEED TO PUT HERE MECHANISM THAT WILL GO BACK AND 
                    #TRY TO PUT ANOTHER POSSIBLE NUMBER IN PREVIOUS CELL THAT WASN'T FILLED ON START
                    #WILL DO IT USING ON_START VARIABLE
                    #print("need to go back")
                    sudoku = restart_next_cells(sudoku, on_start, row, col)
                    if col == 0:
                        if row == 0:
                            #print("error - can't go back")
                            #print("return sudoku")
                            print("Recursion: {}".format(rec))
                            return sudoku
                        else:
                            col = 8
                            row = row - 1
                    else:
                        col = col - 1
                    #print("return solve(sudoku = {}, row = {}, col = {}, number = {}, go_back = {}".format(sudoku,row,col,0,True))
                    return solve(sudoku, on_start, row, col, 0, go_back=True,rec = rec + 1)
                #if number is not 9 (less than 9) recursive solve with number = number + 1
                else:
                    #print("return solve(sudoku = {}, row = {}, col = {}, number = {}, go_back = {}".format(sudoku,row,col,number,False))
                    return solve(sudoku, on_start, row, col, number, go_back=False,rec = rec + 1)
    #when going back cell filled on start just go back
    elif go_back == True and on_start[row][col]==1:
        if col == 0:
            if row == 0:
                #print("error - can't go back")
                #print("return sudoku")
                print("Recursion: {}".format(rec))
                return sudoku
            else:
                col = 8
                row = row - 1
        else:
            col = col - 1
        #print("return solve(sudoku = {}, row = {}, col = {}, number = {}, go_back = {}".format(sudoku,row,col,0,True))
        return solve(sudoku, on_start, row, col, 0, go_back=True,rec = rec + 1)
    #if not going back
    elif go_back == False:
        #if cell = 0 then start checking possibilities
        if on_start[row][col] == 0:
            if_possib = if_possible(sudoku, row, col, number)
            #if possible to put number in cell - put number in that cell
            if if_possib == True:
                sudoku[row][col] = number
                #####
                #if last column
                if col == 8:
                    #if last row
                    if row == 8:
                        #we are in last cell and it is not equal to 0 -> return sudoku
                        #print("return sudoku")
                        print("Recursion: {}".format(rec))
                        return sudoku
                    #if not last row
                    else:
                        #go to firs (0) column and next row
                        col = 0
                        row = row + 1
                #if not last column
                else:
                    #go to next column and stay in same row
                    col = col + 1
                #GO TO NEXT CELL #recursive solve with actual state of sudoku, changed row and col and number 1 - because checking values in new cell starts from 1
                #print("return solve(sudoku = {}, row = {}, col = {}, number = {}, go_back = {}".format(sudoku,row,col,1,False))
                return solve(sudoku, on_start, row, col, 1, go_back=False,rec = rec + 1)
                #####
            #if not possible to put number in cell
            else:
                #if number is 9 - error
                if number == 9:
                    #INSTEAD OF ERROR NOW I NEED TO PUT HERE MECHANISM THAT WILL GO BACK AND 
                    #TRY TO PUT ANOTHER POSSIBLE NUMBER IN PREVIOUS CELL THAT WASN'T FILLED ON START
                    #WILL DO IT USING ON_START VARIABLE
                    #print("need to go back")
                    sudoku = restart_next_cells(sudoku, on_start, row, col)
                    if col == 0:
                        if row == 0:
                            #print("error - can't go back")
                            #print("return sudoku")
                            print("Recursion: {}".format(rec))
                            return sudoku
                        else:
                            col = 8
                            row = row - 1
                    else:
                        col = col - 1
                    #print("return solve(sudoku = {}, row = {}, col = {}, number = {}, go_back = {}".format(sudoku,row,col,0,True))
                    return solve(sudoku, on_start, row, col, 0, go_back=True,rec = rec + 1)
                #if number is not 9 (less than 9) recursive solve with number = number + 1
                else:
                    #print("return solve(sudoku = {}, row = {}, col = {}, number = {}, go_back = {}".format(sudoku,row,col,number + 1,False))
                    return solve(sudoku, on_start, row, col, number + 1, go_back=False,rec = rec + 1)
        #if cell != 0 then don't check possibilities
        else:
            #####
            #if last column
            if col == 8:
                #if last row
                if row == 8:
                    #we are in last cell and it is not equal to 0 -> return sudoku
                    #print("return sudoku")
                    print("Recursion: {}".format(rec))
                    return sudoku
                #if not last row
                else:
                    #go to firs (0) column and next row
                    col = 0
                    row = row + 1
            #if not last column
            else:
                #go to next column and stay in same row
                col = col + 1
            #GO TO NEXT CELL #recursive solve with actual state of sudoku, changed row and col and number 1 - because checking values in new cell starts from 1
            #print("return solve(sudoku = {}, row = {}, col = {}, number = {}, go_back = {}".format(sudoku,row,col,1,False))
            return solve(sudoku, on_start, row, col, 1, go_back=False,rec = rec + 1)
            #####
    
    

In [12]:
sudoku = clear_sudoku()
#sudoku = example_easy_sudoku()
#sudoku = example_hard_sudoku() #works with threading
#sudoku = example_very_hard_sudoku() #doesn't work for now

In [13]:
sudoku

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [14]:
#on_start is array which have 1 in cells filled no start and 0 it cells not filled
on_start = np.copy(sudoku)
np.place(on_start, on_start>0, [True])
print(on_start)
#print(sudoku)

[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]


In [15]:
# for easier sudoku
#%%time
#sudoku = solve(sudoku, on_start, 0, 0, 1, go_back=False, rec=0)

In [16]:
sudoku

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [17]:
threading.stack_size(67108864) # 64MB stack
sys.setrecursionlimit(2 ** 20) # something real big
                               # you actually hit the 64MB limit first
                               # going by other answers, could just use 2**32-1

In [18]:
# for harder sudoku
#%%time
# only new threads get the redefined stack size
thread = threading.Thread(target=solve, args = (sudoku, on_start, 0, 0, 1, False, 0, ))
thread.start()

Recursion: 0
Recursion: 1000
Recursion: 2000
Recursion: 3000
Recursion: 3437


In [19]:
sudoku

array([[1, 2, 3, 4, 5, 6, 7, 8, 9],
       [4, 5, 6, 7, 8, 9, 1, 2, 3],
       [7, 8, 9, 1, 2, 3, 4, 5, 6],
       [2, 1, 4, 3, 6, 5, 8, 9, 7],
       [3, 6, 5, 8, 9, 7, 2, 1, 4],
       [8, 9, 7, 2, 1, 4, 3, 6, 5],
       [5, 3, 1, 6, 4, 2, 9, 7, 8],
       [6, 4, 2, 9, 7, 8, 5, 3, 1],
       [9, 7, 8, 5, 3, 1, 6, 4, 2]])